출처: https://wikidocs.net/262586

## 도구 호출 에이전트(Tool Calling Agent)

In [1]:
from dotenv import load_dotenv
# from langchain_teddynote import logging
from langsmith import traceable
import os

load_dotenv()
# logging.langsmith("agents")
os.environ["LANGCHAIN_PROJECT"] = "agents"


```python
# You can set the project name for a specific tracer instance:
from langchain.callbacks.tracers import LangChainTracer

tracer = LangChainTracer(project_name="My Project")
chain.invoke({"query": "How many people live in canada as of 2023?"}, config={"callbacks": [tracer]})


# LangChain python also supports a context manager for tracing a specific block of code.
# You can set the project name using the project_name parameter.
from langchain_core.tracers.context import tracing_v2_enabled
with tracing_v2_enabled(project_name="My Project"):
    chain.invoke({"query": "How many people live in canada as of 2023?"})

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
("system", "You are a helpful AI."),
("user", "{input}")
])
chat_model = ChatOpenAI()
output_parser = StrOutputParser()

# Tags and metadata can be configured with RunnableConfig
chain = (prompt | chat_model | output_parser).with_config({"tags": ["top-level-tag"], "metadata": {"top-level-key": "top-level-value"}})

# Tags and metadata can also be passed at runtime
chain.invoke({"input": "What is the meaning of life?"}, {"tags": ["shared-tags"], "metadata": {"shared-key": "shared-value"}})
```

In [ ]:
# from dotenv import load_dotenv

import requests
from lxml import html
from langchain.tools import tool

# load_dotenv()

def get_context(link):
    response = requests.get(link)
    response.content
    return ''

@tool
def query_news(query):
    """
    Search Naver News by input keyword
    """
    params = {
        'where': 'news',
        'ie': 'utf8',
        'sm': 'nws_hty',
        'query': query,
    }
    response = requests.get('https://search.naver.com/search.naver', params=params)
    if response.status_code != 200:
        print(response)
        return {}, {}
    news = {} # 번호: [{링크, 제목}, ...]
    press_info = {} # 언론사: 번호
    sections = html.fromstring(response.content).xpath('//*[@id="main_pack"]/section//*[contains(@class, "news_area")]')
    for section in sections:
        items = section.xpath("div")
        info = items[0].xpath('div[2]/a')
        press_name = info[0].text_content().strip()
        link = [i.attrib.get('href', '') for i in info if 'naver' in i.attrib.get('href', '')]
        if link:
            link = link[0]
        else:
            link = 'article/'
        press_num = link.split('article/')[1][:3]
        title = [i.attrib.get('title', '') for i in items[1].xpath('a') if i.attrib.get('title', '')][0]
        if press_name not in press_info:
            press_info[press_name] = press_num
            news[press_num or press_name] = []
        # news[press_num or press_name].append({"link": link, "title": title})
        context = get_context(link) if 'naver' in link else items[1].xpath('div')[0].text_content()
        news[press_num or press_name].append({"link": link, "title": title, "context": context})
    # return news, press_info
    return sum([[{'url': j['link'], 'content': j['title']} for j in i] for i in news.values()], [])


In [ ]:
import requests
from lxml import html
from langchain.tools import tool

@tool
def search_news(query):
    """
    Search Naver News by input keyword
    """
    params = {
        'where': 'news',
        'ie': 'utf8',
        'sm': 'nws_hty',
        'query': query,
    }
    response = requests.get('https://search.naver.com/search.naver', params=params)
    if response.status_code != 200:
        print(response)
        return {}, {}
    news = {} # 번호: [{링크, 제목}, ...]
    press_info = {} # 언론사: 번호
    sections = html.fromstring(response.content).xpath('//*[@id="main_pack"]/section//*[contains(@class, "news_area")]')
    for section in sections:
        items = section.xpath("div")
        info = items[0].xpath('div[2]/a')
        press_name = info[0].text_content().strip()
        link = [i.attrib.get('href', '') for i in info if 'naver' in i.attrib.get('href', '')]
        if link:
            link = link[0]
        else:
            link = 'article/'
        press_num = link.split('article/')[1][:3]
        title = [i.attrib.get('title', '') for i in items[1].xpath('a') if i.attrib.get('title', '')][0]
        if press_name not in press_info:
            press_info[press_name] = press_num
            news[press_num or press_name] = []
        news[press_num or press_name].append({"link": link, "content": title})
    # return news, press_info
    result = sum(news, [])
    return result

In [ ]:
from langchain.tools import tool
from typing import Annotated
from langchain_experimental.utilities import PythonREPL


# 도구 생성
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result


print(f"도구 이름: {query_news.name}")
print(f"도구 설명: {query_news.description}")
print(f"도구 이름: {python_repl_tool.name}")
print(f"도구 설명: {python_repl_tool.description}")

# 도구 이름: search_news
# 도구 설명: Search Google News by input keyword
# 도구 이름: python_repl_tool
# 도구 설명: Use this to execute python code. If you want to see the output of a value,
#           you should print it out with `print(...)`. This is visible to the user.

도구 이름: query_news
도구 설명: query news
[{"url": "...", "content": "..."}]
도구 이름: python_repl_tool
도구 설명: Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user.


In [ ]:
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews

# 도구 생성
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")

도구 이름: search_news
도구 설명: Search Google News by input keyword


In [5]:
# tools 정의
tools = [query_news, python_repl_tool]

- chat_history : 이전 대화 내용을 저장하는 변수 (멀티턴을 지원하지 않는다면, 생략 가능합니다.)
- agent_scratchpad : 에이전트가 임시로 저장하는 변수
- input : 사용자의 입력

In [6]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `query_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Agent 생성
agent = (create_tool_calling_agent(llm, tools, prompt))


## AgentExecutor

AgentExecutor는 도구를 사용하는 에이전트를 실행하는 클래스입니다.

주요 속성
- agent: 실행 루프의 각 단계에서 계획을 생성하고 행동을 결정하는 에이전트
- tools: 에이전트가 사용할 수 있는 유효한 도구 목록
- return_intermediate_steps: 최종 출력과 함께 에이전트의 중간 단계 경로를 반환할지 여부
- max_iterations: 실행 루프를 종료하기 전 최대 단계 수
- max_execution_time: 실행 루프에 소요될 수 있는 최대 시간
- early_stopping_method: 에이전트가 AgentFinish를 반환하지 않을 때 사용할 조기 종료 방법. ("force" or "generate")
- "force" 는 시간 또는 반복 제한에 도달하여 중지되었다는 문자열을 반환합니다.
- "generate" 는 에이전트의 LLM 체인을 마지막으로 한 번 호출하여 이전 단계에 따라 최종 답변을 생성합니다.
- handle_parsing_errors: 에이전트의 출력 파서에서 발생한 오류 처리 방법. (True, False, 또는 오류 처리 함수)
- trim_intermediate_steps: 중간 단계를 트리밍하는 방법. (-1 trim 하지 않음, 또는 트리밍 함수)

주요 메서드
- invoke: 에이전트 실행
- stream: 최종 출력에 도달하는 데 필요한 단계를 스트리밍

주요 기능
- 도구 검증: 에이전트와 호환되는 도구인지 확인
- 실행 제어: 최대 반복 횟수 및 실행 시간 제한 설정 가능
- 오류 처리: 출력 파싱 오류에 대한 다양한 처리 옵션 제공
- 중간 단계 관리: 중간 단계 트리밍 및 반환 옵션
- 비동기 지원: 비동기 실행 및 스트리밍 지원

최적화 팁
- max_iterations와 max_execution_time을 적절히 설정하여 실행 시간 관리
- trim_intermediate_steps를 활용하여 메모리 사용량 최적화
- 복잡한 작업의 경우 stream 메서드를 사용하여 단계별 결과 모니터링

In [ ]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=10,
    max_execution_time=10,
    handle_parsing_errors=True,
)

@traceable
def run_agent_executor(query):
    return agent_executor.invoke({"input": query})

query = "AI 투자와 관련된 뉴스를 검색해 주세요."
result = run_agent_executor(query)
print("Agent 실행 결과:")
print(result["output"])


In [ ]:
from langchain.agents import AgentExecutor

def run_agent_executor(agent, tools, query):
    # AgentExecutor 생성
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        max_iterations=10,
        max_execution_time=10,
        handle_parsing_errors=True,
    )

    # AgentExecutor 실행
    result = agent_executor.invoke({"input": query})
    return result

query = "AI 투자와 관련된 뉴스를 검색해 주세요."
result = run_agent_executor(agent, tools, query)
print("Agent 실행 결과:")
print(result["output"])


Failed to use model_dump to serialize <class 'langchain_core.runnables.base.RunnableSequence'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_core.runnables.base.RunnableLambda'>)
Failed to use model_dump to serialize <class 'langchain_core.tools.structured.StructuredTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)
Failed to use model_dump to serialize <class 'langchain_core.tools.structured.StructuredTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)




> Entering new AgentExecutor chain...

Invoking: `query_news` with `{'query': 'AI 투자'}`


[{'url': 'https://n.news.naver.com/mnews/article/003/0012948733?sid=104', 'content': '한화 금융 3사, \'AI 메카\' 美샌프란 전초기지 출범…"투자기회 창출"'}, {'url': 'https://n.news.naver.com/mnews/article/421/0007955820?sid=101', 'content': "금융권, 생성형 AI 활용 'AI 은행원' 서비스 선보인다"}, {'url': 'https://n.news.naver.com/mnews/article/014/0005279310?sid=101', 'content': '한화 금융 3사, 미국 샌프란시스코 AI 센터 개소'}, {'url': 'https://n.news.naver.com/mnews/article/277/0005513690?sid=101', 'content': '한화 금융 3사, 美 샌프란시스코 AI 센터 개소'}, {'url': 'https://n.news.naver.com/mnews/article/243/0000069291?sid=101', 'content': '“생성형 AI가 대출 상담도 뚝딱” 우리은행, 금융권 첫 사례'}, {'url': 'https://n.news.naver.com/mnews/article/092/0002355685?sid=105', 'content': '에이베러, 캡스톤파트너스서 10억 투자 유치'}, {'url': 'https://n.news.naver.com/mnews/article/021/0002676838?sid=101', 'content': 'MS “내년은 AI가 일상·업무 필수 될 것”'}, {'url': 'https://n.news.naver.com/mnews/article/009/0005410374?sid=101',

## Stream 출력으로 단계별 결과 확인

|출력	|내용|
|---------------|------------|
|Action	        |actions: AgentAction 또는 그 하위 클래스|
|               |messages: 액션 호출에 해당하는 채팅 메시지|
|Observation	|steps: 현재 액션과 그 관찰을 포함한 에이전트가 지금까지 수행한 작업의 기록|
|               |messages: 함수 호출 결과(즉, 관찰)를 포함한 채팅 메시지|
|Final Answer	|output: AgentFinish|
|               |messages: 최종 출력을 포함한 채팅 메시지|

In [ ]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True,
)

# 스트리밍 모드 실행
result = agent_executor.stream({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."}) 

for step in result:
    # 중간 단계 출력
    print(step)
